In [ ]:
# Статистические тесты

In [2]:
import pandas as pd # для работы с таблицами
from scipy import stats 
import statsmodels

In [4]:
# прочитаем данные
data = pd.read_csv('https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv')
# дадим названия колонкам
data.rename(columns={'Making Unit 1': 'Пиццерия №1', 'Making Unit 2': 'Пиццерия №2'}, inplace=True)
data.head()

,Пиццерия №1,Пиццерия №2
0,6.8090,6.7703
1,6.4376,7.5093
2,6.9157,6.7300
3,7.3012,6.7878
4,7.4488,7.1522


Мы будем использовать информацию о диаметре пицц разных производителей (содержимое ячеек)

## Проверка данных на нормальность

In [5]:
# cформулируем гипотезу:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (отвергаем H0)'

# Установим уровень значимости:
alpha = 0.05

### Тест Шапиро-Уилка

In [9]:
from scipy.stats import shapiro

_, p = shapiro(data)
print('p=%.3f' % p) # Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.204
Данные распределены нормально


### Тест Д’Агостино

In [7]:
from scipy.stats import normaltest

_, p = normaltest(data)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0]) # Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.251
Данные распределены нормально


Вывод: данные распределены нормально. Приступаем к поиску зависимостей.

Зависимая переменная (диаметр пиццы) - количественная.
Мы рассматриваем 2 разные совокупности (Пиццерия №1 и Пиццерия №2)
В каждой совокупности больше 35 значений.
Значит, нам нужет Т-тест

## Независимый Т-тест

In [11]:
# сформулируем гипотезу
H0 = 'Нет значимой разницы между диаметрами пиццы в разных пиццериях.'
Ha = 'Есть значимая разница между диаметрами пиццы в разных пиццериях.'

# уровень значимости остается прежним

In [12]:
from scipy.stats import ttest_ind

def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(data['Пиццерия №1'], data['Пиццерия №2'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(data)


*** Результаты независимого T-теста ***
0.47 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет значимой разницы между диаметрами пиццы в разных пиццериях.


-----

## Корреляция Спирмена

In [14]:
from numpy.random import rand
from scipy.stats import spearmanr

# создадим 2 массива с сильной зависимостью
data1 = rand(1000) * 20
data2 = data1 + (rand(1000) * 10)

corr, p = spearmanr(data1, data2)
print(corr,p)

0.9079153999154 0.0


р показывает вероятность того, что из нескоррелированных данных можно создать датасет с таким же коэффициентом корреляции, что и в переданном.
Нулевая гипотеза по умолчанию - связь между переменными отсутствует

In [15]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.0 <= 0.05. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.


-----

## ANOVA тест

Однофакторный дисперсионный анализ ANOVA проверяет нулевую гипотезу о том, что две или более групп имеют одинаковое среднее значение генеральной совокупности.
В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

In [17]:
from scipy.stats import f_oneway

petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

H0 = 'Нет значимой разницы между средним размером раковины мидий, выращенных в трех разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий, выращенных в трех разных местах.'

_, p = f_oneway(petersburg, magadan, tvarminne)

if p>alpha:
  print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
  print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.007870900547143829 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий, выращенных в трех разных местах.


-----                                                                   

Возьмем другой датасет в работу.
Он содержит информацию по давлению 120 человек

In [18]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv')
data.head()

,patient,sex,agegrp,bp_before,bp_after
0,1,Male,30-45,143,153
1,2,Male,30-45,163,170
2,3,Male,30-45,153,168
3,4,Male,30-45,153,142
4,5,Male,30-45,146,141


In [19]:
print('Размер выборки: ', data.shape)

Размер выборки:  (120, 5)


Тест на нормальность распределения интересующих нас признаков (bp_before и bp_after)

In [21]:
# cформулируем гипотезу:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (отвергаем H0)'

# Установим уровень значимости:
alpha = 0.05

from scipy.stats import shapiro

_, p = shapiro(data['bp_before'])
print('p=%.3f' % p) # отображение значения р

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.000
Данные не распределены нормально (отвергаем H0)


In [22]:
_, p = shapiro(data['bp_after'])
print('p=%.3f' % p) # отображение значения р

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.020
Данные не распределены нормально (отвергаем H0)


## Z-тест

In [24]:
from statsmodels.stats import weightstats

_ ,p = weightstats.ztest(data['bp_before'], x2=data['bp_after'], value=0,alternative='two-sided')
print(float(p))

# напоминаю, нулевая гипотеза по умолчанию - различия между данными отсутствуют

if p < 0.05:
    print("Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу об отсутсвии различий между данными.")

0.002162306611369422
Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.


## Z-тест на пропорцию

Z-критерий одной пропорции используется для сравнения наблюдаемой пропорции с теоретической.

В этом тесте используются следующие нулевые гипотезы:

$H_0: p = p_0$ (доля мужчин, предоставивших данные о своем давлении, равна гипотетической пропорции $p_0$)

Альтернативная гипотеза может быть двусторонней, левосторонней или правосторонней:

$H_1 (двусторонний): p ≠ p_0$ (доля мужчин не равна некоторому гипотетическому значению $p_0$)
$H_1 (левосторонний): p < p_0$ (доля мужчин меньше некоторого гипотетического значения $p_0$)
$H_1 (правосторонний): p > p_0$ (доля мужчин больше некоторого гипотетического значения $p_0$)

Предположим, что доля мужчин в нашем датачете равна 50%.

$p_0$: гипотетическая доля мужчин = 0.50

$x$: число мужчин в выборке мужчин: `len(data[data.sex == 'Male'])`

$n$: размер выборки = `len(data)`

Покажем, как использовать функцию `proportions_ztest` для выполнения  **z-теста**:

In [25]:
p_0 = 0.5
n = len(data)
x = len(data[data.sex == 'Male'])

from statsmodels.stats.proportion import proportions_ztest

_, p = proportions_ztest(count=x, nobs=n, value=p_0)

if p < 0.05:
    print("Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу.")

Мы не можем отвергнуть нулевую гипотезу.
